In [1]:
# load and the test weekly fpl scrape
season="2019-20"
gw=21
model_type="rf"

In [2]:
import os
import pandas as pd
import numpy as np
from keras.models import load_model
import pickle
from sklearn.externals import joblib

Using TensorFlow backend.
/usr/local/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [3]:
os.chdir(os.path.join(os.getcwd(), "..", "..", "data"));

In [4]:
# scaler = joblib.load("scaler.save")

In [5]:
player_id_df = pd.read_csv("player_id_map.csv");

In [6]:
team_id_df = pd.read_csv("team_id_map.csv");

In [7]:
X_test = pd.read_csv("x-test.csv");

In [8]:
os.chdir(os.path.join(os.getcwd(), ".."))

In [9]:
model_dir = os.path.join(os.getcwd(), "model", season, str(gw))

In [10]:
lineup_dir = os.path.join(os.getcwd(), "predicted-lineup", season, str(gw))

In [11]:
os.chdir(model_dir)

In [12]:
if model_type == "nn":
    model = load_model("{}-model.h5".format(model_type))
else:
    with open("{}-model.pkl".format(model_type), 'rb') as f:
            model = pickle.load(f)

In [13]:
os.chdir(lineup_dir)

In [14]:
weekly_scrape = pd.read_csv("gw{}-fpl-data.csv".format(gw));

In [15]:
weekly_scrape_scaled = weekly_scrape[X_test.columns];
# scaler.transform(weekly_scrape[X_test.columns]);

In [16]:
y_pred_weekly = model.predict(weekly_scrape_scaled);

In [17]:
y_pred_weekly_df = pd.DataFrame({'player_id': list(weekly_scrape["player_id"]), 
                                 'position': list(weekly_scrape["position"]),
                                 'predicted': y_pred_weekly.flatten()});

In [18]:
player_names = [];
opponent_names = [];

for index, row in weekly_scrape.iterrows():
    player_id = row["player_id"];
    opponent_id = row["opponent_id"];
    
    player_names.append(player_id_df[player_id_df["player_id"] == player_id]["actual_name"].iloc[0]);
    opponent_names.append(team_id_df[team_id_df["team_id"] == opponent_id]["team_name"].iloc[0]);

In [19]:
y_pred_weekly_df["player_name"] = player_names;
y_pred_weekly_df["opponent"] = opponent_names;

In [20]:
y_pred_weekly_df = y_pred_weekly_df[["player_id", "position", "player_name", "opponent", "predicted"]];

In [21]:
y_pred_weekly_df.sort_values(["predicted"], ascending=False)

,player_id,position,player_name,opponent,predicted
7,585,MID,Sadio Mané,Sheffield United,3.424835
2,648,DEF,Trent Alexander-Arnold,Sheffield United,3.277376
12,161,FWD,Danny Ings,Tottenham Hotspur,3.062190
14,442,FWD,Marcus Rashford,Arsenal,3.001257
9,385,MID,Kevin De Bruyne,Everton,2.945523
15,61,MID,Anthony Martial,Arsenal,2.939882
0,3,GK,Aaron Ramsdale,West Ham United,2.723894
4,977,DEF,Issa Diop,Bournemouth,2.559108
16,882,FWD,Tammy Abraham,Brighton and Hove Albion,2.491733
5,451,DEF,Martin Kelly,Norwich City,2.468180


In [22]:
y_pred_weekly_df.to_csv("gw{}-predicted-data-{}.csv".format(gw, model_type));

In [23]:
min_max_pos = {
    "GK": {
        "min": 1,
        "max": 1
    },
    "DEF": {
        "min": 3,
        "max": 5
    },
    "MID": {
        "min": 2,
        "max": 5
    },
    "FWD": {
        "min": 1,
        "max": 3
    }
}

In [24]:
def best11(df, max_players=11, verbose=0):
    selectedTeam = pd.DataFrame(columns=df.columns);
    positions = list(np.unique(df["position"]));
    availablePlayers = pd.DataFrame(columns=df.columns);
    
    if (df.shape[0] < max_players):
        raise Exception("Err: Cannot have less than {} players in a game".format(max_players));
    
    # base cases - min 1 keeper, 3 def, 2 mids, 1 fwd
    for position in positions:
        original_query = df[df["position"] == position].sort_values(["predicted"], ascending=False);
        if (original_query.shape[0] < min_max_pos[position]["min"]):
            raise Exception("Err: Cannot have less than {} {} in the formation. Found only {}".format(min_max_pos[position]["min"], position, original_query.shape[0]));
        df_head = original_query.head(min_max_pos[position]["min"]);
        df_tail = original_query.tail(original_query.shape[0] - df_head.shape[0]);
        selectedTeam = selectedTeam.append(df_head);
        availablePlayers = availablePlayers.append(df_tail);

    gk_ind = positions.index("GK");
    del positions[gk_ind];
    
    num_players = selectedTeam.shape[0];
    
    if verbose == 1:
        print("Initial selected team", selectedTeam)
        print("\n")
    
    # recursive case - max 5 def, 5 mids, 3 fwd
    while num_players < max_players:
        num_def = selectedTeam[selectedTeam["position"] == "DEF"].shape[0];
        num_mid = selectedTeam[selectedTeam["position"] == "MID"].shape[0];
        num_fwd = selectedTeam[selectedTeam["position"] == "FWD"].shape[0];
        
        candidates=[]

        # for each position, select the best candidate
        for position in positions:
            original_query = df[df["position"] == position].sort_values(["predicted"], ascending=False);
            num_position = selectedTeam[selectedTeam["position"] == position].shape[0];
            
            # and availablePlayers[availablePlayers["position"] == position].shape[0] > 0
            if (num_position < min_max_pos[position]["max"] and availablePlayers[availablePlayers["position"] == position].shape[0] > 0):
                candidate = original_query.head(selectedTeam[selectedTeam["position"] == position].shape[0] + 1).tail(1)
                candidates.append(candidate);
        
        if verbose == 1:
            print("Candidates: ", candidates);
        
        max_score = 0
        for index in range(len(candidates)):
            score = candidates[index]["predicted"].iloc[0]
            if (score > max_score):
                max_score = score
                best_player = candidates[index];
        
        if verbose == 1:
            print("Candidate chosen: ", best_player.iloc[0]["player_name"])
        selectedTeam = selectedTeam.append(best_player);
        availablePlayers = availablePlayers[availablePlayers["player_id"] != best_player.iloc[0]["player_id"]];
        
        num_players = num_players + 1;
        
        if verbose == 1:
            print("Chose {} players".format(num_players))
            print(selectedTeam)
            print("\n")
        
    best11 = pd.DataFrame(columns=selectedTeam.columns);
    best11 = best11.append(selectedTeam[selectedTeam["position"] == "GK"]);
    best11 = best11.append(selectedTeam[selectedTeam["position"] == "DEF"]);
    best11 = best11.append(selectedTeam[selectedTeam["position"] == "MID"]);
    best11 = best11.append(selectedTeam[selectedTeam["position"] == "FWD"]);
    
    top2_ = list(best11.sort_values(["predicted"], ascending=False).head(2).index);
    best11["roles"] = ""
    best11.at[top2_[0], 'roles']="C"
    best11.at[top2_[1], 'roles']="VC"
    
    best11["predicted"] = best11["predicted"].apply(lambda predicted: round(predicted, 0));
    
    best11["predicted"] = best11.apply(lambda row: row["predicted"] * 2 if row["roles"] == "C" else row["predicted"], axis=1)
    
    return best11


In [25]:
best11 = best11(y_pred_weekly_df)

In [26]:
best11

,player_id,position,player_name,opponent,predicted,roles
0,3,GK,Aaron Ramsdale,West Ham United,3.0,
2,648,DEF,Trent Alexander-Arnold,Sheffield United,3.0,VC
4,977,DEF,Issa Diop,Bournemouth,3.0,
5,451,DEF,Martin Kelly,Norwich City,2.0,
7,585,MID,Sadio Mané,Sheffield United,6.0,C
9,385,MID,Kevin De Bruyne,Everton,3.0,
15,61,MID,Anthony Martial,Arsenal,3.0,
8,78,MID,Bamidele Alli,Southampton,2.0,
12,161,FWD,Danny Ings,Tottenham Hotspur,3.0,
14,442,FWD,Marcus Rashford,Arsenal,3.0,


In [27]:
sum(best11["predicted"])

33.0

In [28]:
best11.to_csv("gw{}-best11-{}.csv".format(gw,model_type), index=False);